In [ ]:
from magpi.prelude import *
from magpi.r_fun import r0, sphere, translate
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.colors import LightSource

%matplotlib widget

In [ ]:
# create Pacman ADF from R0 system

unit_vec = lambda x: x / jnp.linalg.norm(x, axis=-1, keepdims=True)
mouth = r0.intersection(
    lambda x: jnp.dot(x, unit_vec(jnp.array([1., 0., -1.8]))),
    lambda x: jnp.dot(x, unit_vec(jnp.array([1., 0., 1.8])))
)
left_eye = translate(sphere(0.25), [0.4, -0.5, 0.6])
right_eye = translate(sphere(0.25), [0.4, 0.5, 0.6])
pacman = r0.difference(sphere(1.), mouth)
pacman = r0.union(pacman, left_eye)
pacman = r0.union(pacman, right_eye)
pacman = r0.union(pacman, translate(sphere(0.15), [1., 0., 0]))
pacman = r0.union(pacman, translate(sphere(0.15), [2., 0., 0]))
pacman = jax.jit(pacman)

In [ ]:
resolution = 50
X = jnp.meshgrid(
    jnp.linspace(-1.5, 2.5, resolution),
    jnp.linspace(-1.5, 1.5, resolution),
    jnp.linspace(-1.5, 1.5, resolution),
)
X = jnp.stack(X, axis=-1)
Z = jnp.apply_along_axis(pacman, -1, X)
Z = np.array(Z)
verts, faces, normals, values = measure.marching_cubes(
    Z, 0., 
    spacing=(1 / resolution, 1 / resolution, 1 / resolution)
)

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')

ls = LightSource(-180, -120)
mesh = Poly3DCollection(
    verts[faces], 
    alpha=1.0,
    shade=True,
    lightsource=ls,
    facecolors="yellow",
)

ax.add_collection3d(mesh)


ax.set_xlim(0., 1)
ax.set_ylim(0.2, 0.8)
ax.set_zlim(0.2, 0.8)
ax.view_init(10, 50)
plt.axis("off")
plt.tight_layout()